In [16]:
from bs4 import BeautifulSoup, NavigableString
import requests
from pathlib import Path

import pandas as pd

In [17]:
BASE_URL = "http://www.olympedia.org/editions/61/medal"

In [18]:
data_values = []

In [19]:
html = requests.get(BASE_URL).text

In [20]:
soup = BeautifulSoup(html, "html.parser")

In [21]:
data_table = soup.find("table")

MEDALS = {1: "Gold", 3: "Silver", 5: "Bronze"}
#main_sports = {}
main_sports = []
main_sport = None
for row in data_table.find_all("tr"):
    h2 = row.find("h2")
    if h2:
        main_sport = h2.get_text()
        #main_sports[main_sport] = []
    elif main_sport:
        cols = row.find_all("td")
        event = cols[0].get_text()
        
        event = event.split(", ")
        event_name = ", ".join(event[:-1])
        event_name_gender = event[-1]
        
        """event = event.split(", ")
        if(len(event) > 2):
            event_name = event[0] + " " + event[1]
            event_name_gender = event[-1]
        else:    
            event_name = event[0]
            if(event[1] =="Men" or event[1] == "Woman"):
                event_name_gender = event[1]
            else:
                event_name_gender = event[-1]"""
        
        
        
        for i in (1, 3, 5):
            winners = [
                link.get_text()
                for link in cols[i].find_all("a")
                if "athlete" in link.get("href") 
            ]
            if not winners:
                winners = [cols[i].get_text()]

            # flags = [link.get("src") for link in cols[i + 1].find_all("img")]
            # country_codes = [Path(flag).stem for flag in flags]
            
            country_codes = []
            flags = []
            country_links = [
                link
                for link in cols[i + 1].find_all("a")
                if "countries" in link.get("href")
            ]
            for link in country_links:
                country_codes.append(link.get_text())
                img = link.find("img")
                if img:
                    flags.append(img.get("src"))
                        

            for winner, flag, country_code in zip(winners, flags,country_codes):
                #new_entry = [event, MEDALS[i], winner, flag, country_code]
                #main_sports[main_sport].append(new_entry)
                new_entry = (event_name, event_name_gender, MEDALS[i], winner, flag, country_code, main_sport)
                main_sports.append(new_entry)


In [22]:
#main_sport = "Artistic Gymnastics"
#for event in main_sports[main_sport]:
#    print(event)

In [23]:
tokyo_2020_df = pd.DataFrame(main_sports)

In [24]:
tokyo_2020_df.columns = ['Category', 'Category_Gender' ,'Medal', 'Athlete_Full_Name', 'Country_IMG_URL', 'Country_Code', 'Sport_Event']

In [25]:
tokyo_2020_df

,Category,Category_Gender,Medal,Athlete_Full_Name,Country_IMG_URL,Country_Code,Sport_Event
0,3x3 Basketball,Men,Gold,Latvia,/images/flags/LAT.png,LAT,3x3 Basketball
1,3x3 Basketball,Men,Silver,Russian Olympic Committee,/images/flags/ROC.png,ROC,3x3 Basketball
2,3x3 Basketball,Men,Bronze,Serbia,/images/flags/SRB.png,SRB,3x3 Basketball
3,3x3 Basketball,Women,Gold,United States,/images/flags/USA.png,USA,3x3 Basketball
4,3x3 Basketball,Women,Silver,Russian Olympic Committee,/images/flags/ROC.png,ROC,3x3 Basketball
...,...,...,...,...,...,...,...
941,"Light-Heavyweight, Freestyle",Women,Bronze,Meerim Zhumanazarova,/images/flags/KGZ.png,KGZ,Wrestling
942,"Heavyweight, Freestyle",Women,Gold,Aline Rotter-Focken,/images/flags/GER.png,GER,Wrestling
943,"Heavyweight, Freestyle",Women,Silver,Adeline Gray,/images/flags/USA.png,USA,Wrestling
944,"Heavyweight, Freestyle",Women,Bronze,Yasemin Adar,/images/flags/TUR.png,TUR,Wrestling


In [26]:
tokyo_2020_df["Country_IMG_URL"] = "https://www.olympedia.org" + tokyo_2020_df["Country_IMG_URL"]

In [27]:
tokyo_2020_df

,Category,Category_Gender,Medal,Athlete_Full_Name,Country_IMG_URL,Country_Code,Sport_Event
0,3x3 Basketball,Men,Gold,Latvia,https://www.olympedia.org/images/flags/LAT.png,LAT,3x3 Basketball
1,3x3 Basketball,Men,Silver,Russian Olympic Committee,https://www.olympedia.org/images/flags/ROC.png,ROC,3x3 Basketball
2,3x3 Basketball,Men,Bronze,Serbia,https://www.olympedia.org/images/flags/SRB.png,SRB,3x3 Basketball
3,3x3 Basketball,Women,Gold,United States,https://www.olympedia.org/images/flags/USA.png,USA,3x3 Basketball
4,3x3 Basketball,Women,Silver,Russian Olympic Committee,https://www.olympedia.org/images/flags/ROC.png,ROC,3x3 Basketball
...,...,...,...,...,...,...,...
941,"Light-Heavyweight, Freestyle",Women,Bronze,Meerim Zhumanazarova,https://www.olympedia.org/images/flags/KGZ.png,KGZ,Wrestling
942,"Heavyweight, Freestyle",Women,Gold,Aline Rotter-Focken,https://www.olympedia.org/images/flags/GER.png,GER,Wrestling
943,"Heavyweight, Freestyle",Women,Silver,Adeline Gray,https://www.olympedia.org/images/flags/USA.png,USA,Wrestling
944,"Heavyweight, Freestyle",Women,Bronze,Yasemin Adar,https://www.olympedia.org/images/flags/TUR.png,TUR,Wrestling


In [28]:
tokyo_2020_df = tokyo_2020_df.reset_index()

In [29]:
tokyo_2020_df[tokyo_2020_df["Sport_Event"] == "Equestrian Dressage"]

,index,Category,Category_Gender,Medal,Athlete_Full_Name,Country_IMG_URL,Country_Code,Sport_Event
344,344,Individual,Open,Gold,Jessica Von Bredow-Werndl,https://www.olympedia.org/images/flags/GER.png,GER,Equestrian Dressage
345,345,Individual,Open,Silver,Isabell Werth,https://www.olympedia.org/images/flags/GER.png,GER,Equestrian Dressage
346,346,Individual,Open,Bronze,Charlotte Dujardin,https://www.olympedia.org/images/flags/GBR.png,GBR,Equestrian Dressage
347,347,Team,Open,Gold,Germany,https://www.olympedia.org/images/flags/GER.png,GER,Equestrian Dressage
348,348,Team,Open,Silver,United States,https://www.olympedia.org/images/flags/USA.png,USA,Equestrian Dressage
349,349,Team,Open,Bronze,Great Britain,https://www.olympedia.org/images/flags/GBR.png,GBR,Equestrian Dressage


In [30]:
tokyo_2020_df["Category_Gender"].unique()

array(['Men', 'Women', 'Mixed', 'Open'], dtype=object)

In [31]:
tokyo_2020_df.to_csv("files/tokyo_2020.csv")